In [1]:
from bs4 import BeautifulSoup
import re 
import requests
import pandas as pd
from tqdm.notebook import tqdm
import random
from random import randint
import warnings
warnings.filterwarnings("ignore")

In [2]:
url = "https://www.billboard.com/charts/hot-100"
# downloading  html with a get request
response = requests.get(url)
# check response status code 
response.status_code

200

In [3]:
billboardsoup = BeautifulSoup(response.content, 'html.parser')



In [4]:
top_100 = len(billboardsoup.select('h3.c-title.a-no-trucate'))
song = []
artist = []

for i in tqdm(range(top_100)):
    song.append(billboardsoup.select('h3.c-title.a-no-trucate')[i].get_text(strip=True))
    artist.append(billboardsoup.select('span.c-label.a-font-primary-s')[i].get_text(strip=True))

  0%|          | 0/100 [00:00<?, ?it/s]

In [5]:
billboard_top100 = pd.DataFrame({'song':song,'artist':artist})

In [6]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import getpass
import pprint

In [7]:
client_id = str(getpass.getpass('client_id: '))
client_secret = str(getpass.getpass('client_secret: '))
# embedding getpass credentials in the access key
sp = spotipy.Spotify(auth_manager = SpotifyClientCredentials(client_id = client_id, client_secret = client_secret))

client_id: ········
client_secret: ········


In [8]:
first_playlist = sp.user_playlist_tracks("solitude collective", "1IEQ8C3G1qT0W80muYgROT")


In [9]:
def get_tracks(user_id, playlist_id):
    results = sp.user_playlist_tracks(user_id, playlist_id)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks
first_results = get_tracks("solitude collective", "1IEQ8C3G1qT0W80muYgROT")


In [10]:
song_name = []
song_uri = []
song_id = []
song_artists = []
playlist = first_results
for r in first_results:
    song_name.append(r['track']['name'])
    song_uri.append(r['track']['uri'])
    song_id.append(r['track']['id'])
    song_artists.append(r['track']['artists'][0]['name'])
playlist_info = pd.DataFrame({'song_name':song_name, 'song_id':song_id, 'artist':song_artists})

In [11]:
features_info = pd.DataFrame(sp.audio_features(tracks = song_id[0:99]))
features_info2 = pd.DataFrame(sp.audio_features(tracks = song_id[99:199]))
features_info3 = pd.DataFrame(sp.audio_features(tracks = song_id[199:299]))
features_info4 = pd.DataFrame(sp.audio_features(tracks = song_id[299:399]))
features_info5 = pd.DataFrame(sp.audio_features(tracks = song_id[399:]))
features = pd.concat([features_info, features_info2, features_info3, features_info4, features_info5])
features.reset_index(drop=True, inplace=True)


In [12]:
first_playlist = pd.concat([playlist_info, features], axis=1)
first_playlist.reset_index()



,index,song_name,song_id,artist,danceability,energy,key,loudness,mode,speechiness,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0,Carry on Wayward Son,2SUpC3UgKwLVOS2FtZif9N,Kansas,0.452,0.788,7,-5.712,1,0.0326,...,0.4410,0.7930,126.739,audio_features,2SUpC3UgKwLVOS2FtZif9N,spotify:track:2SUpC3UgKwLVOS2FtZif9N,https://api.spotify.com/v1/tracks/2SUpC3UgKwLV...,https://api.spotify.com/v1/audio-analysis/2SUp...,322093,4
1,1,Jump,2PfBSkCKZDXMsJogMT0wML,Van Halen,0.547,0.822,0,-7.201,1,0.0292,...,0.0645,0.7740,130.138,audio_features,2PfBSkCKZDXMsJogMT0wML,spotify:track:2PfBSkCKZDXMsJogMT0wML,https://api.spotify.com/v1/tracks/2PfBSkCKZDXM...,https://api.spotify.com/v1/audio-analysis/2PfB...,239560,4
2,2,Rock You Like A Hurricane,0XIvZ82aDF7JiSi3ZE320u,Scorpions,0.482,0.601,7,-12.846,1,0.0486,...,0.2490,0.7740,125.619,audio_features,0XIvZ82aDF7JiSi3ZE320u,spotify:track:0XIvZ82aDF7JiSi3ZE320u,https://api.spotify.com/v1/tracks/0XIvZ82aDF7J...,https://api.spotify.com/v1/audio-analysis/0XIv...,252493,4
3,3,Back In Black,08mG3Y1vljYA6bvDt4Wqkj,AC/DC,0.310,0.700,9,-5.678,1,0.0470,...,0.0828,0.7630,188.386,audio_features,08mG3Y1vljYA6bvDt4Wqkj,spotify:track:08mG3Y1vljYA6bvDt4Wqkj,https://api.spotify.com/v1/tracks/08mG3Y1vljYA...,https://api.spotify.com/v1/audio-analysis/08mG...,255493,4
4,4,Hot Blooded,6xMHglHoafdDFGXS6qfwSH,Foreigner,0.705,0.806,0,-4.830,1,0.0814,...,0.0676,0.7630,117.945,audio_features,6xMHglHoafdDFGXS6qfwSH,spotify:track:6xMHglHoafdDFGXS6qfwSH,https://api.spotify.com/v1/tracks/6xMHglHoafdD...,https://api.spotify.com/v1/audio-analysis/6xMH...,268693,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
423,423,I'd Be Lost If It Wasn't for You,64FhrwziLZYXXTtmhgk9Xh,Elijah Honey,0.639,0.743,5,-7.198,1,0.0527,...,0.3950,0.8490,120.415,audio_features,64FhrwziLZYXXTtmhgk9Xh,spotify:track:64FhrwziLZYXXTtmhgk9Xh,https://api.spotify.com/v1/tracks/64FhrwziLZYX...,https://api.spotify.com/v1/audio-analysis/64Fh...,181893,4
424,424,My Curse,6zKF4293k44ItKWJJgrhXv,Killswitch Engage,0.197,0.956,10,-2.908,0,0.0663,...,0.2610,0.2090,177.018,audio_features,6zKF4293k44ItKWJJgrhXv,spotify:track:6zKF4293k44ItKWJJgrhXv,https://api.spotify.com/v1/tracks/6zKF4293k44I...,https://api.spotify.com/v1/audio-analysis/6zKF...,244773,3
425,425,Livin' On A Prayer,7qQnBfwXrw2tZNFG4Uf57N,Bon Jovi,0.520,0.887,0,-3.296,1,0.0349,...,0.1860,0.7930,122.528,audio_features,7qQnBfwXrw2tZNFG4Uf57N,spotify:track:7qQnBfwXrw2tZNFG4Uf57N,https://api.spotify.com/v1/tracks/7qQnBfwXrw2t...,https://api.spotify.com/v1/audio-analysis/7qQn...,250627,4
426,426,"Touch-A, Touch-A, Touch Me",2SK1kGbpABI9EyUbADSP5z,Susan Sarandon,0.355,0.351,1,-13.572,1,0.0621,...,0.1090,0.4310,171.896,audio_features,2SK1kGbpABI9EyUbADSP5z,spotify:track:2SK1kGbpABI9EyUbADSP5z,https://api.spotify.com/v1/tracks/2SK1kGbpABI9...,https://api.spotify.com/v1/audio-analysis/2SK1...,151922,4


In [13]:
second_playlist = sp.user_playlist_tracks("Spotify", "37i9dQZF1DX4UtSsGT1Sbe")


In [14]:
def get_tracks(user_id, playlist_id):
    results = sp.user_playlist_tracks(user_id, playlist_id)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks
second_results = get_tracks("Spotify", "37i9dQZF1DX4UtSsGT1Sbe")

In [15]:
song_name = []
song_uri = []
song_id = []
song_artists = []
playlist = second_results
for r in second_results:
    song_name.append(r['track']['name'])
    song_uri.append(r['track']['uri'])
    song_id.append(r['track']['id'])
    song_artists.append(r['track']['artists'][0]['name'])
playlist2_info = pd.DataFrame({'song_name':song_name, 'song_id':song_id, 'artist':song_artists})

In [16]:
features2_info = pd.DataFrame(sp.audio_features(tracks = song_id[0:99]))
features2_info2 = pd.DataFrame(sp.audio_features(tracks = song_id[99:]))
features2 = pd.concat([features2_info, features2_info2])
features2.reset_index(drop=True, inplace=True)

In [17]:
second_playlist = pd.concat([playlist2_info, features2], axis=1)
second_playlist.reset_index()


,index,song_name,song_id,artist,danceability,energy,key,loudness,mode,speechiness,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0,I Wanna Dance with Somebody (Who Loves Me),2tUBqZG2AbRi7Q0BIrVrEj,Whitney Houston,0.709,0.824,1,-8.824,1,0.0453,...,0.0888,0.867,118.818,audio_features,2tUBqZG2AbRi7Q0BIrVrEj,spotify:track:2tUBqZG2AbRi7Q0BIrVrEj,https://api.spotify.com/v1/tracks/2tUBqZG2AbRi...,https://api.spotify.com/v1/audio-analysis/2tUB...,291293,4
1,1,Billie Jean,5ChkMS8OtdzJeqyybCc9R5,Michael Jackson,0.920,0.654,11,-3.051,0,0.0401,...,0.0360,0.847,117.046,audio_features,5ChkMS8OtdzJeqyybCc9R5,spotify:track:5ChkMS8OtdzJeqyybCc9R5,https://api.spotify.com/v1/tracks/5ChkMS8OtdzJ...,https://api.spotify.com/v1/audio-analysis/5Chk...,293827,4
2,2,Time After Time,1Jj6MF0xDOMA3Ut2Z368Bx,Cyndi Lauper,0.724,0.436,0,-9.321,1,0.0282,...,0.0908,0.324,130.439,audio_features,1Jj6MF0xDOMA3Ut2Z368Bx,spotify:track:1Jj6MF0xDOMA3Ut2Z368Bx,https://api.spotify.com/v1/tracks/1Jj6MF0xDOMA...,https://api.spotify.com/v1/audio-analysis/1Jj6...,243067,4
3,3,Take on Me,2WfaOiMkCvy7F5fcp2zZ8L,a-ha,0.573,0.902,6,-7.638,0,0.0540,...,0.0928,0.876,84.412,audio_features,2WfaOiMkCvy7F5fcp2zZ8L,spotify:track:2WfaOiMkCvy7F5fcp2zZ8L,https://api.spotify.com/v1/tracks/2WfaOiMkCvy7...,https://api.spotify.com/v1/audio-analysis/2Wfa...,225280,4
4,4,"Footloose - From ""Footloose"" Soundtrack",2vz1CsL5WBsbpBcwgboTAw,Kenny Loggins,0.583,0.905,2,-6.132,1,0.0733,...,0.0593,0.646,173.982,audio_features,2vz1CsL5WBsbpBcwgboTAw,spotify:track:2vz1CsL5WBsbpBcwgboTAw,https://api.spotify.com/v1/tracks/2vz1CsL5WBsb...,https://api.spotify.com/v1/audio-analysis/2vz1...,226827,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,145,I'm Gonna Be (500 Miles),66S14BkJDxgkYxLl5DCqOz,The Proclaimers,0.851,0.551,4,-5.177,1,0.0396,...,0.0827,0.807,131.922,audio_features,66S14BkJDxgkYxLl5DCqOz,spotify:track:66S14BkJDxgkYxLl5DCqOz,https://api.spotify.com/v1/tracks/66S14BkJDxgk...,https://api.spotify.com/v1/audio-analysis/66S1...,219467,4
146,146,Missing You,1Qrdlkgg9I4J7r3P4kZNwr,John Waite,0.552,0.552,11,-9.736,0,0.0364,...,0.0460,0.593,208.571,audio_features,1Qrdlkgg9I4J7r3P4kZNwr,spotify:track:1Qrdlkgg9I4J7r3P4kZNwr,https://api.spotify.com/v1/tracks/1Qrdlkgg9I4J...,https://api.spotify.com/v1/audio-analysis/1Qrd...,269760,4
147,147,Sexual Healing,3VZmChrnVW8JK6ano4gSED,Marvin Gaye,0.805,0.602,0,-9.700,0,0.0579,...,0.0581,0.744,94.382,audio_features,3VZmChrnVW8JK6ano4gSED,spotify:track:3VZmChrnVW8JK6ano4gSED,https://api.spotify.com/v1/tracks/3VZmChrnVW8J...,https://api.spotify.com/v1/audio-analysis/3VZm...,238600,4
148,148,Golden Brown,2AX5E86cn9n2dgioZEjirI,The Stranglers,0.562,0.383,6,-15.474,1,0.0304,...,0.1380,0.599,93.764,audio_features,2AX5E86cn9n2dgioZEjirI,spotify:track:2AX5E86cn9n2dgioZEjirI,https://api.spotify.com/v1/tracks/2AX5E86cn9n2...,https://api.spotify.com/v1/audio-analysis/2AX5...,206760,3


In [18]:
third_playlist = sp.user_playlist_tracks("Peter Nordestgaard", "0lNwm5xFEgRx4iDP2DLx3j")

In [19]:
def get_tracks(user_id, playlist_id):
    results = sp.user_playlist_tracks(user_id, playlist_id)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks
third_results = get_tracks("Peter Nordestgaard", "0lNwm5xFEgRx4iDP2DLx3j")


In [20]:


song_name = []
song_uri = []
song_id = []
song_artists = []
playlist = third_results
for r in third_results:
    song_name.append(r['track']['name'])
    song_uri.append(r['track']['uri'])
    song_id.append(r['track']['id'])
    song_artists.append(r['track']['artists'][0]['name'])
playlist3_info = pd.DataFrame({'song_name':song_name, 'song_id':song_id, 'artist':song_artists})

In [21]:
features3_info = pd.DataFrame(sp.audio_features(tracks = song_id[0:99]))
features3_info1 = pd.DataFrame(sp.audio_features(tracks = song_id[99:199]))
features3_info2 = pd.DataFrame(sp.audio_features(tracks = song_id[199:299]))
features3_info3 = pd.DataFrame(sp.audio_features(tracks = song_id[299:399]))
features3_info4 = pd.DataFrame(sp.audio_features(tracks = song_id[399:499]))
features3_info5 = pd.DataFrame(sp.audio_features(tracks = song_id[499:599]))
features3_info6 = pd.DataFrame(sp.audio_features(tracks = song_id[599:699]))
features3_info7 = pd.DataFrame(sp.audio_features(tracks = song_id[699:799]))
features3_info8 = pd.DataFrame(sp.audio_features(tracks = song_id[799:899]))
features3_info9 = pd.DataFrame(sp.audio_features(tracks = song_id[899:999]))
features3_info10 = pd.DataFrame(sp.audio_features(tracks = song_id[999:]))
features3 = pd.concat([features3_info, features3_info1, features3_info2, features3_info3, features3_info4, features3_info5, features3_info6, features3_info7, features3_info8, features3_info9, features3_info10])
features3.reset_index(drop=True, inplace=True)



In [22]:
third_playlist = pd.concat([playlist3_info, features3_info], axis=1)
third_playlist.reset_index()


,index,song_name,song_id,artist,danceability,energy,key,loudness,mode,speechiness,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0,Imagine - Remastered 2010,7pKfPomDEeI4TPT6EOYjn9,John Lennon,0.547,0.257,0.0,-12.358,1.0,0.0252,...,0.0935,0.169,75.752,audio_features,7pKfPomDEeI4TPT6EOYjn9,spotify:track:7pKfPomDEeI4TPT6EOYjn9,https://api.spotify.com/v1/tracks/7pKfPomDEeI4...,https://api.spotify.com/v1/audio-analysis/7pKf...,187867.0,4.0
1,1,A Whiter Shade Of Pale,3qitymULqEibr7yknRMKU4,Procol Harum,0.249,0.660,0.0,-6.905,1.0,0.0342,...,0.0891,0.435,149.813,audio_features,3qitymULqEibr7yknRMKU4,spotify:track:3qitymULqEibr7yknRMKU4,https://api.spotify.com/v1/tracks/3qitymULqEib...,https://api.spotify.com/v1/audio-analysis/3qit...,248947.0,4.0
2,2,My Sweet Lord,1chu8wc9zbNqTQ2ahROys1,George Harrison,0.537,0.685,1.0,-8.935,0.0,0.0333,...,0.0839,0.542,120.965,audio_features,1chu8wc9zbNqTQ2ahROys1,spotify:track:1chu8wc9zbNqTQ2ahROys1,https://api.spotify.com/v1/tracks/1chu8wc9zbNq...,https://api.spotify.com/v1/audio-analysis/1chu...,281227.0,4.0
3,3,God Only Knows - Mono / 1997 Remastered,6iGU74CwXuT4XVepjc9Emf,The Beach Boys,0.521,0.487,6.0,-7.525,0.0,0.0299,...,0.0579,0.483,117.072,audio_features,6iGU74CwXuT4XVepjc9Emf,spotify:track:6iGU74CwXuT4XVepjc9Emf,https://api.spotify.com/v1/tracks/6iGU74CwXuT4...,https://api.spotify.com/v1/audio-analysis/6iGU...,173040.0,4.0
4,4,Bridge Over Troubled Water,6l8EbYRtQMgKOyc1gcDHF9,Simon & Garfunkel,0.149,0.206,3.0,-13.888,1.0,0.0323,...,0.1150,0.264,79.764,audio_features,6l8EbYRtQMgKOyc1gcDHF9,spotify:track:6l8EbYRtQMgKOyc1gcDHF9,https://api.spotify.com/v1/tracks/6l8EbYRtQMgK...,https://api.spotify.com/v1/audio-analysis/6l8E...,293120.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,She Will Be Loved,7sapKrjDij2fpDVj0GxP66,Maroon 5,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
996,996,Achy Breaky Heart,2EoIt9vdgFRNW03u5IvFsQ,Billy Ray Cyrus,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
997,997,Reet Petite,3h4udS0WeWbsur3yfjvnm4,Jackie Wilson,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
998,998,All I Want for Christmas Is You,0bYg9bo50gSsH3LtXe2SQn,Mariah Carey,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
fourth_playlist = sp.user_playlist_tracks("Spotify", "37i9dQZF1DX2UXfvEIZvDK")


In [24]:
def get_tracks(user_id, playlist_id):
    results = sp.user_playlist_tracks(user_id, playlist_id)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks
fourth_results = get_tracks("Spotify", "37i9dQZF1DX2UXfvEIZvDK")

In [25]:
song_name = []
song_uri = []
song_id = []
song_artists = []
playlist = fourth_results
for r in fourth_results:
    song_name.append(r['track']['name'])
    song_uri.append(r['track']['uri'])
    song_id.append(r['track']['id'])
    song_artists.append(r['track']['artists'][0]['name'])
playlist4_info = pd.DataFrame({'song_name':song_name, 'song_id':song_id, 'artist':song_artists})


In [26]:
features4_info = pd.DataFrame(sp.audio_features(tracks = song_id[0:]))


In [27]:
fourth_playlist = pd.concat([playlist4_info, features4_info], axis=1)
fourth_playlist.reset_index()


,index,song_name,song_id,artist,danceability,energy,key,loudness,mode,speechiness,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0,Scar Tissue,1G391cbiT3v3Cywg8T7DM1,Red Hot Chili Peppers,0.595,0.717,0,-4.803,1,0.0295,...,0.1080,0.5470,88.969,audio_features,1G391cbiT3v3Cywg8T7DM1,spotify:track:1G391cbiT3v3Cywg8T7DM1,https://api.spotify.com/v1/tracks/1G391cbiT3v3...,https://api.spotify.com/v1/audio-analysis/1G39...,215907,4
1,1,Santeria,2hnMS47jN0etwvFPzYk11f,Sublime,0.682,0.765,1,-5.021,0,0.0395,...,0.1880,0.5670,90.807,audio_features,2hnMS47jN0etwvFPzYk11f,spotify:track:2hnMS47jN0etwvFPzYk11f,https://api.spotify.com/v1/tracks/2hnMS47jN0et...,https://api.spotify.com/v1/audio-analysis/2hnM...,182747,4
2,2,Island In The Sun,2MLHyLy5z5l5YRp7momlgw,Weezer,0.654,0.810,4,-6.260,0,0.0288,...,0.1650,0.6610,114.623,audio_features,2MLHyLy5z5l5YRp7momlgw,spotify:track:2MLHyLy5z5l5YRp7momlgw,https://api.spotify.com/v1/tracks/2MLHyLy5z5l5...,https://api.spotify.com/v1/audio-analysis/2MLH...,200307,4
3,3,Every Morning,2ouURa1AIXp3AvkS52Jry5,Sugar Ray,0.833,0.679,8,-4.496,1,0.0365,...,0.0769,0.9750,109.916,audio_features,2ouURa1AIXp3AvkS52Jry5,spotify:track:2ouURa1AIXp3AvkS52Jry5,https://api.spotify.com/v1/tracks/2ouURa1AIXp3...,https://api.spotify.com/v1/audio-analysis/2ouU...,219867,4
4,4,WALLS,41QhRnFDdhMLtR8tmnolbr,Kings of Leon,0.560,0.320,8,-14.340,1,0.0313,...,0.1100,0.1070,82.511,audio_features,41QhRnFDdhMLtR8tmnolbr,spotify:track:41QhRnFDdhMLtR8tmnolbr,https://api.spotify.com/v1/tracks/41QhRnFDdhML...,https://api.spotify.com/v1/audio-analysis/41Qh...,329667,4
5,5,Talk Tonight - Remastered,2LNV0347QKt4AHxRSfCBij,Oasis,0.376,0.423,4,-8.499,0,0.0242,...,0.3610,0.2990,85.213,audio_features,2LNV0347QKt4AHxRSfCBij,spotify:track:2LNV0347QKt4AHxRSfCBij,https://api.spotify.com/v1/tracks/2LNV0347QKt4...,https://api.spotify.com/v1/audio-analysis/2LNV...,263747,4
6,6,What It's Like,6vUPPEN2V2lZ78b6DyEIjZ,Everlast,0.679,0.557,0,-7.423,1,0.0281,...,0.0866,0.4050,85.152,audio_features,6vUPPEN2V2lZ78b6DyEIjZ,spotify:track:6vUPPEN2V2lZ78b6DyEIjZ,https://api.spotify.com/v1/tracks/6vUPPEN2V2lZ...,https://api.spotify.com/v1/audio-analysis/6vUP...,303133,4
7,7,Way down We Go,0y1QJc3SJVPKJ1OvFmFqe6,KALEO,0.489,0.505,10,-8.022,0,0.1170,...,0.1040,0.3370,163.255,audio_features,0y1QJc3SJVPKJ1OvFmFqe6,spotify:track:0y1QJc3SJVPKJ1OvFmFqe6,https://api.spotify.com/v1/tracks/0y1QJc3SJVPK...,https://api.spotify.com/v1/audio-analysis/0y1Q...,213707,4
8,8,Make It Wit Chu,0Cgd3CK6D2hhbwmLRvtscH,Queens of the Stone Age,0.704,0.811,9,-5.308,1,0.0277,...,0.1300,0.5780,91.970,audio_features,0Cgd3CK6D2hhbwmLRvtscH,spotify:track:0Cgd3CK6D2hhbwmLRvtscH,https://api.spotify.com/v1/tracks/0Cgd3CK6D2hh...,https://api.spotify.com/v1/audio-analysis/0Cgd...,290493,4
9,9,The Man Who Sold The World,15VRO9CQwMpbqUYA7e6Hwg,Nirvana,0.483,0.753,1,-8.937,0,0.0832,...,0.2570,0.4200,115.102,audio_features,15VRO9CQwMpbqUYA7e6Hwg,spotify:track:15VRO9CQwMpbqUYA7e6Hwg,https://api.spotify.com/v1/tracks/15VRO9CQwMpb...,https://api.spotify.com/v1/audio-analysis/15VR...,261093,4


In [28]:
fifth_playlist = sp.user_playlist_tracks("Spotify", "37i9dQZF1DX8Md3JnnrexB")

In [29]:
def get_tracks(user_id, playlist_id):
    results = sp.user_playlist_tracks(user_id, playlist_id)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks
fifth_results = get_tracks("Spotify", "37i9dQZF1DX8Md3JnnrexB")



In [30]:
song_name = []
song_uri = []
song_id = []
song_artists = []
playlist = fifth_results
for r in fifth_results:
    song_name.append(r['track']['name'])
    song_uri.append(r['track']['uri'])
    song_id.append(r['track']['id'])
    song_artists.append(r['track']['artists'][0]['name'])
playlist5_info = pd.DataFrame({'song_name':song_name, 'song_id':song_id, 'artist':song_artists})


In [31]:
features5_info = pd.DataFrame(sp.audio_features(tracks = song_id[0:]))


HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=1k1Bqnv2R0uJXQN4u6LKYt,745H5CctFr12Mo7cqa1BMH,3zBhihYUHBmGd2bcQIobrF,63xdwScd1Ai1GigAwQxE8y,1Omt5bfz1tZUCqd26HxbS0,3nFJbZCHP4d9vduKjJLdBL,49MHCPzvMLXhRjDantBMVH,3gsCAGsWr6pUm1Vy7CPPob,3l3xTXsUXeWlkPqzMs7mPD,7MF7QAodbGzNYav5ZfIhAY,3VZmChrnVW8JK6ano4gSED,0n2pjCIMKwHSXoYfEbYMfX,3M8FzayQWtkvOhqMn2V4T2,7jmHyHMEqm9MJWiMAneF05,5K09WxKdlkroDyEVyOSySy,1lx8ddGT5wCD6W2xmLeRKG,10vkYRKw6Jjr7try1ir50G,3cfnGXJ9bmiWvFqEO6ff8B,3IDc87lrpEHWFBASmgTgFV,0KOE1hat4SIer491XKk4Pa,6Ro2z4RtAUew9Kz2HZWZUi,1j2LuIf7mv15ZVug1Xy5qz,7IljDTNkQEyy77CcaRbiAj,1ckU1EhAO0Nr73QYw24SWJ,2yCAGizViD7HZXwNYxJToQ,5ZoDwIP1ntHwciLjydJ8X2,23IK8TF2MSiLnuEjaYPfVV,7JOyY9GA3P2Evx50oetDKb,1qfDfZAvrkSAFrAR2FTQDv,73GR50SgGV5hybgWDSFWtD,6lUXneXZ9jc3HSMHgd8oqh,5GvWrvLIqoHroq7YvO260M,02srSkeu2pzybuVr2B9TJm,5HrVgmO1govSOrPUehW1VV,3nsJGZ5RV2ZHkbKvD771gU,39Bd345OWEhRNyfayhp9gv,61sQYdFNS6sEBYCyr1q5gn,2ezqQeBiC72gwMJoO4w1hA,0ACACkoHUwgfgY5CxVIL4N,7gh2v4IHnxdiwSgA6xluhe,0J8A1ZVyJcR

SpotifyException: http status: 400, code:-1 - https://api.spotify.com/v1/audio-features/?ids=1k1Bqnv2R0uJXQN4u6LKYt,745H5CctFr12Mo7cqa1BMH,3zBhihYUHBmGd2bcQIobrF,63xdwScd1Ai1GigAwQxE8y,1Omt5bfz1tZUCqd26HxbS0,3nFJbZCHP4d9vduKjJLdBL,49MHCPzvMLXhRjDantBMVH,3gsCAGsWr6pUm1Vy7CPPob,3l3xTXsUXeWlkPqzMs7mPD,7MF7QAodbGzNYav5ZfIhAY,3VZmChrnVW8JK6ano4gSED,0n2pjCIMKwHSXoYfEbYMfX,3M8FzayQWtkvOhqMn2V4T2,7jmHyHMEqm9MJWiMAneF05,5K09WxKdlkroDyEVyOSySy,1lx8ddGT5wCD6W2xmLeRKG,10vkYRKw6Jjr7try1ir50G,3cfnGXJ9bmiWvFqEO6ff8B,3IDc87lrpEHWFBASmgTgFV,0KOE1hat4SIer491XKk4Pa,6Ro2z4RtAUew9Kz2HZWZUi,1j2LuIf7mv15ZVug1Xy5qz,7IljDTNkQEyy77CcaRbiAj,1ckU1EhAO0Nr73QYw24SWJ,2yCAGizViD7HZXwNYxJToQ,5ZoDwIP1ntHwciLjydJ8X2,23IK8TF2MSiLnuEjaYPfVV,7JOyY9GA3P2Evx50oetDKb,1qfDfZAvrkSAFrAR2FTQDv,73GR50SgGV5hybgWDSFWtD,6lUXneXZ9jc3HSMHgd8oqh,5GvWrvLIqoHroq7YvO260M,02srSkeu2pzybuVr2B9TJm,5HrVgmO1govSOrPUehW1VV,3nsJGZ5RV2ZHkbKvD771gU,39Bd345OWEhRNyfayhp9gv,61sQYdFNS6sEBYCyr1q5gn,2ezqQeBiC72gwMJoO4w1hA,0ACACkoHUwgfgY5CxVIL4N,7gh2v4IHnx

In [32]:
fifth_playlist = pd.concat([playlist5_info, features5_info], axis=1)
fifth_playlist.reset_index()


NameError: name 'features5_info' is not defined

In [33]:
sixth_playlist = sp.user_playlist_tracks("Spotify", "37i9dQZF1DXcgZcN2HVMoe")


In [34]:

def get_tracks(user_id, playlist_id):
    results = sp.user_playlist_tracks(user_id, playlist_id)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks
sixth_results = get_tracks("Spotify", "37i9dQZF1DXcgZcN2HVMoe")


In [35]:
song_name = []
song_uri = []
song_id = []
song_artists = []
playlist = sixth_results
for r in sixth_results:
    song_name.append(r['track']['name'])
    song_uri.append(r['track']['uri'])
    song_id.append(r['track']['id'])
    song_artists.append(r['track']['artists'][0]['name'])
playlist6_info = pd.DataFrame({'song_name':song_name, 'song_id':song_id, 'artist':song_artists})


In [36]:
features6_info = pd.DataFrame(sp.audio_features(tracks = song_id[0:99]))
features6_info2 = pd.DataFrame(sp.audio_features(tracks = song_id[99:199]))
features6_info3 = pd.DataFrame(sp.audio_features(tracks = song_id[199:]))
features6 = pd.concat([features6_info, features6_info2, features6_info3])
features6.reset_index(drop=True, inplace=True)

In [37]:
sixth_playlist = pd.concat([playlist6_info, features6], axis=1)
sixth_playlist.reset_index()


,index,song_name,song_id,artist,danceability,energy,key,loudness,mode,speechiness,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0,Until I Found You,0T5iIrXA4p5GsubkhuBIKV,Stephen Sanchez,0.539,0.508,10,-6.050,1,0.0288,...,0.1740,0.2270,101.358,audio_features,0T5iIrXA4p5GsubkhuBIKV,spotify:track:0T5iIrXA4p5GsubkhuBIKV,https://api.spotify.com/v1/tracks/0T5iIrXA4p5G...,https://api.spotify.com/v1/audio-analysis/0T5i...,177720,3
1,1,TV,3GYlZ7tbxLOxe6ewMNVTkw,Billie Eilish,0.412,0.236,4,-15.159,0,0.0379,...,0.1470,0.1190,140.214,audio_features,3GYlZ7tbxLOxe6ewMNVTkw,spotify:track:3GYlZ7tbxLOxe6ewMNVTkw,https://api.spotify.com/v1/tracks/3GYlZ7tbxLOx...,https://api.spotify.com/v1/audio-analysis/3GYl...,281381,4
2,2,Coastline,4sDbdBjuYQVdNNB4PiHUPo,Hollow Coves,0.670,0.496,2,-11.230,1,0.0277,...,0.0584,0.4270,100.170,audio_features,4sDbdBjuYQVdNNB4PiHUPo,spotify:track:4sDbdBjuYQVdNNB4PiHUPo,https://api.spotify.com/v1/tracks/4sDbdBjuYQVd...,https://api.spotify.com/v1/audio-analysis/4sDb...,233973,4
3,3,Beyond,1Omt5bfz1tZUCqd26HxbS0,Leon Bridges,0.581,0.687,6,-5.400,1,0.1050,...,0.1090,0.1870,76.014,audio_features,1Omt5bfz1tZUCqd26HxbS0,spotify:track:1Omt5bfz1tZUCqd26HxbS0,https://api.spotify.com/v1/tracks/1Omt5bfz1tZU...,https://api.spotify.com/v1/audio-analysis/1Omt...,240600,4
4,4,Hold My Girl,42bbDWZ8WmXTH7PkYAlGLu,George Ezra,0.614,0.527,10,-6.374,1,0.0302,...,0.1100,0.3540,121.941,audio_features,42bbDWZ8WmXTH7PkYAlGLu,spotify:track:42bbDWZ8WmXTH7PkYAlGLu,https://api.spotify.com/v1/tracks/42bbDWZ8WmXT...,https://api.spotify.com/v1/audio-analysis/42bb...,211764,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,195,Dakota,2LOEgB0Jk9suqAP5dDS4bp,Tom Speight,0.621,0.296,3,-9.801,1,0.0291,...,0.1030,0.2570,121.918,audio_features,2LOEgB0Jk9suqAP5dDS4bp,spotify:track:2LOEgB0Jk9suqAP5dDS4bp,https://api.spotify.com/v1/tracks/2LOEgB0Jk9su...,https://api.spotify.com/v1/audio-analysis/2LOE...,294499,4
196,196,Fireworks,6qIUd9qdVd3xtVYT6CAQZn,Hazlett,0.693,0.388,9,-11.328,1,0.0338,...,0.1070,0.0616,134.005,audio_features,6qIUd9qdVd3xtVYT6CAQZn,spotify:track:6qIUd9qdVd3xtVYT6CAQZn,https://api.spotify.com/v1/tracks/6qIUd9qdVd3x...,https://api.spotify.com/v1/audio-analysis/6qIU...,242015,4
197,197,Blinding Lights,66aMnwdhkEc1V0AAnyYchX,Teddy Swims,0.512,0.543,0,-6.915,0,0.0310,...,0.1120,0.4420,159.900,audio_features,66aMnwdhkEc1V0AAnyYchX,spotify:track:66aMnwdhkEc1V0AAnyYchX,https://api.spotify.com/v1/tracks/66aMnwdhkEc1...,https://api.spotify.com/v1/audio-analysis/66aM...,214875,4
198,198,Microdose,1EdPgv2qJv1wu1zHsFfR1v,John Craigie,0.676,0.479,5,-11.438,1,0.0280,...,0.0631,0.3230,106.018,audio_features,1EdPgv2qJv1wu1zHsFfR1v,spotify:track:1EdPgv2qJv1wu1zHsFfR1v,https://api.spotify.com/v1/tracks/1EdPgv2qJv1w...,https://api.spotify.com/v1/audio-analysis/1EdP...,335860,4


In [38]:

seventh_playlist = sp.user_playlist_tracks("Arian", "5dxn0i8MPl6XFVVxNatd6U")



In [39]:
def get_tracks(user_id, playlist_id):
    results = sp.user_playlist_tracks(user_id, playlist_id)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks
seventh_results = get_tracks("Arian", "5dxn0i8MPl6XFVVxNatd6U")



In [40]:
song_name = []
song_uri = []
song_id = []
song_artists = []
playlist = seventh_results
for r in seventh_results:
    song_name.append(r['track']['name'])
    song_uri.append(r['track']['uri'])
    song_id.append(r['track']['id'])
    song_artists.append(r['track']['artists'][0]['name'])
playlist7_info = pd.DataFrame({'song_name':song_name, 'song_id':song_id, 'artist':song_artists})


In [41]:
features7_info = pd.DataFrame(sp.audio_features(tracks = song_id[0:99]))
features7_info2 = pd.DataFrame(sp.audio_features(tracks = song_id[99:199]))
features7_info3 = pd.DataFrame(sp.audio_features(tracks = song_id[199:299]))
features7_info4 = pd.DataFrame(sp.audio_features(tracks = song_id[299:399]))
features7_info5 = pd.DataFrame(sp.audio_features(tracks = song_id[399:499]))
features7_info6 = pd.DataFrame(sp.audio_features(tracks = song_id[499:]))
features7 = pd.concat([features7_info, features7_info2, features7_info3, features7_info4, features7_info5, features7_info6])
features7.reset_index(drop=True, inplace=True)


In [42]:

seventh_playlist = pd.concat([playlist7_info, features7], axis=1)
seventh_playlist.reset_index()


,index,song_name,song_id,artist,danceability,energy,key,loudness,mode,speechiness,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0,Respect,7s25THrKz86DM225dOYwnr,Aretha Franklin,0.805,0.558,0,-5.226,1,0.0410,...,0.0546,0.965,114.950,audio_features,7s25THrKz86DM225dOYwnr,spotify:track:7s25THrKz86DM225dOYwnr,https://api.spotify.com/v1/tracks/7s25THrKz86D...,https://api.spotify.com/v1/audio-analysis/7s25...,147600,4
1,1,Fight The Power,1yo16b3u0lptm6Cs7lx4AD,Public Enemy,0.797,0.582,2,-12.970,1,0.2550,...,0.5170,0.415,105.974,audio_features,1yo16b3u0lptm6Cs7lx4AD,spotify:track:1yo16b3u0lptm6Cs7lx4AD,https://api.spotify.com/v1/tracks/1yo16b3u0lpt...,https://api.spotify.com/v1/audio-analysis/1yo1...,282640,4
2,2,A Change Is Gonna Come,0KOE1hat4SIer491XKk4Pa,Sam Cooke,0.212,0.383,10,-10.071,1,0.0347,...,0.2990,0.452,173.790,audio_features,0KOE1hat4SIer491XKk4Pa,spotify:track:0KOE1hat4SIer491XKk4Pa,https://api.spotify.com/v1/tracks/0KOE1hat4SIe...,https://api.spotify.com/v1/audio-analysis/0KOE...,191160,3
3,3,Like a Rolling Stone,3AhXZa8sUQht0UEdBJgpGc,Bob Dylan,0.482,0.721,0,-6.839,1,0.0321,...,0.1890,0.557,95.263,audio_features,3AhXZa8sUQht0UEdBJgpGc,spotify:track:3AhXZa8sUQht0UEdBJgpGc,https://api.spotify.com/v1/tracks/3AhXZa8sUQht...,https://api.spotify.com/v1/audio-analysis/3AhX...,369600,4
4,4,Smells Like Teen Spirit,5ghIJDpPoe3CfHMGu71E6T,Nirvana,0.502,0.912,1,-4.556,1,0.0564,...,0.1060,0.720,116.761,audio_features,5ghIJDpPoe3CfHMGu71E6T,spotify:track:5ghIJDpPoe3CfHMGu71E6T,https://api.spotify.com/v1/tracks/5ghIJDpPoe3C...,https://api.spotify.com/v1/audio-analysis/5ghI...,301920,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,495,Without You,6MrILERJLA3Jcq0E9poZsp,Harry Nilsson,0.381,0.186,4,-15.477,1,0.0290,...,0.0694,0.142,65.058,audio_features,6MrILERJLA3Jcq0E9poZsp,spotify:track:6MrILERJLA3Jcq0E9poZsp,https://api.spotify.com/v1/tracks/6MrILERJLA3J...,https://api.spotify.com/v1/audio-analysis/6MrI...,201000,4
496,496,Truth Hurts,3HWzoMvoF3TQfYg4UPszDq,Lizzo,0.715,0.624,4,-3.046,0,0.1140,...,0.1230,0.412,158.087,audio_features,3HWzoMvoF3TQfYg4UPszDq,spotify:track:3HWzoMvoF3TQfYg4UPszDq,https://api.spotify.com/v1/tracks/3HWzoMvoF3TQ...,https://api.spotify.com/v1/audio-analysis/3HWz...,173325,4
497,497,Pancho and Lefty,42Wb5T7YmdrzVD81cUWWAg,Townes Van Zandt,0.257,0.453,2,-8.063,1,0.0305,...,0.5640,0.397,148.848,audio_features,42Wb5T7YmdrzVD81cUWWAg,spotify:track:42Wb5T7YmdrzVD81cUWWAg,https://api.spotify.com/v1/tracks/42Wb5T7Ymdrz...,https://api.spotify.com/v1/audio-analysis/42Wb...,241080,4
498,498,Baby Love,5uES1C2NgkdrNHiCwf9jRr,The Supremes,0.595,0.643,5,-9.275,1,0.0365,...,0.5380,0.730,135.633,audio_features,5uES1C2NgkdrNHiCwf9jRr,spotify:track:5uES1C2NgkdrNHiCwf9jRr,https://api.spotify.com/v1/tracks/5uES1C2Ngkdr...,https://api.spotify.com/v1/audio-analysis/5uES...,158040,4


In [43]:
eighth_playlist = sp.user_playlist_tracks("Marcel", "6OAF6gkxAe4lqHdSLmGpdd")

In [44]:
def get_tracks(user_id, playlist_id):
    results = sp.user_playlist_tracks(user_id, playlist_id)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks
eighth_results = get_tracks("Marcel", "6OAF6gkxAe4lqHdSLmGpdd")

In [45]:

song_name = []
song_uri = []
song_id = []
song_artists = []
playlist = eighth_results
for r in eighth_results:
    song_name.append(r['track']['name'])
    song_uri.append(r['track']['uri'])
    song_id.append(r['track']['id'])
    song_artists.append(r['track']['artists'][0]['name'])
playlist8_info = pd.DataFrame({'song_name':song_name, 'song_id':song_id, 'artist':song_artists})

In [46]:
features8_info = pd.DataFrame(sp.audio_features(tracks = song_id[0:99]))
features8_info2 = pd.DataFrame(sp.audio_features(tracks = song_id[99:199]))
features8_info3 = pd.DataFrame(sp.audio_features(tracks = song_id[199:299]))
features8_info4 = pd.DataFrame(sp.audio_features(tracks = song_id[299:399]))
features8_info5 = pd.DataFrame(sp.audio_features(tracks = song_id[399:499]))
features8_info6 = pd.DataFrame(sp.audio_features(tracks = song_id[499:599]))
features8_info7 = pd.DataFrame(sp.audio_features(tracks = song_id[599:699]))
features8_info8 = pd.DataFrame(sp.audio_features(tracks = song_id[699:799]))
features8_info9 = pd.DataFrame(sp.audio_features(tracks = song_id[799:899]))
features8_info10 = pd.DataFrame(sp.audio_features(tracks = song_id[899:999]))
features8_info11 = pd.DataFrame(sp.audio_features(tracks = song_id[999:]))
features8 = pd.concat([features8_info, features8_info2, features8_info3, features8_info4, features8_info5, features8_info6, features8_info7, features8_info8, features8_info9, features8_info10, features8_info11])
features8.reset_index(drop=True, inplace=True)

In [47]:
eighth_playlist = pd.concat([playlist8_info, features8], axis=1)
eighth_playlist.reset_index()

,index,song_name,song_id,artist,danceability,energy,key,loudness,mode,speechiness,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0,I'm Still Standing,5fwP61Nmo5xXYIjO9XEWqb,Elton John,0.504,0.904,6,-6.863,1,0.1790,...,0.1400,0.772,176.808,audio_features,5fwP61Nmo5xXYIjO9XEWqb,spotify:track:5fwP61Nmo5xXYIjO9XEWqb,https://api.spotify.com/v1/tracks/5fwP61Nmo5xX...,https://api.spotify.com/v1/audio-analysis/5fwP...,183440,4
1,1,Le Freak,28NBmftocOzTPEb6OYA9fW,CHIC,0.867,0.746,7,-7.445,1,0.0475,...,0.1230,0.933,118.616,audio_features,28NBmftocOzTPEb6OYA9fW,spotify:track:28NBmftocOzTPEb6OYA9fW,https://api.spotify.com/v1/tracks/28NBmftocOzT...,https://api.spotify.com/v1/audio-analysis/28NB...,327573,4
2,2,I Feel Love,6efRrfjV1YW1rtecLUHoP1,Donna Summer,0.690,0.675,0,-16.247,1,0.0433,...,0.0987,0.940,128.100,audio_features,6efRrfjV1YW1rtecLUHoP1,spotify:track:6efRrfjV1YW1rtecLUHoP1,https://api.spotify.com/v1/tracks/6efRrfjV1YW1...,https://api.spotify.com/v1/audio-analysis/6efR...,493600,4
3,3,Blueprint,1RuzwWdidBXXMVpwoBBIL1,Rainbirds,0.475,0.757,0,-12.237,0,0.0451,...,0.1470,0.715,194.193,audio_features,1RuzwWdidBXXMVpwoBBIL1,spotify:track:1RuzwWdidBXXMVpwoBBIL1,https://api.spotify.com/v1/tracks/1RuzwWdidBXX...,https://api.spotify.com/v1/audio-analysis/1Ruz...,331467,4
4,4,Life Is For Living,6OAVhdZQs1KMNH1TfXbFJE,Barclay James Harvest,0.700,0.565,7,-14.761,1,0.0823,...,0.0724,0.919,103.109,audio_features,6OAVhdZQs1KMNH1TfXbFJE,spotify:track:6OAVhdZQs1KMNH1TfXbFJE,https://api.spotify.com/v1/tracks/6OAVhdZQs1KM...,https://api.spotify.com/v1/audio-analysis/6OAV...,218933,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1077,1077,My Baby Just Cares for Me - 2013 Remastered Ve...,6VTbbVjKOC2qWagIDbkJrC,Nina Simone,0.746,0.243,4,-15.134,0,0.1230,...,0.0905,0.425,118.578,audio_features,6VTbbVjKOC2qWagIDbkJrC,spotify:track:6VTbbVjKOC2qWagIDbkJrC,https://api.spotify.com/v1/tracks/6VTbbVjKOC2q...,https://api.spotify.com/v1/audio-analysis/6VTb...,215747,4
1078,1078,Do You Wanna Dance,21J0jPiMlIpXpKVFjyARP4,Cliff Richard & The Shadows,0.648,0.639,7,-15.287,1,0.0564,...,0.1080,0.953,84.164,audio_features,21J0jPiMlIpXpKVFjyARP4,spotify:track:21J0jPiMlIpXpKVFjyARP4,https://api.spotify.com/v1/tracks/21J0jPiMlIpX...,https://api.spotify.com/v1/audio-analysis/21J0...,135600,4
1079,1079,Working My Way Back to You,7cLkQkC9Z7hnN767rfRl5G,The Spinners,0.818,0.768,9,-6.975,0,0.0482,...,0.1070,0.922,121.562,audio_features,7cLkQkC9Z7hnN767rfRl5G,spotify:track:7cLkQkC9Z7hnN767rfRl5G,https://api.spotify.com/v1/tracks/7cLkQkC9Z7hn...,https://api.spotify.com/v1/audio-analysis/7cLk...,242667,4
1080,1080,I Can Hear Your Heartbeat,7n0lp1r1yHYXN2jZvqzbAI,Chris Rea,0.689,0.722,7,-8.575,1,0.0313,...,0.0488,0.970,123.145,audio_features,7n0lp1r1yHYXN2jZvqzbAI,spotify:track:7n0lp1r1yHYXN2jZvqzbAI,https://api.spotify.com/v1/tracks/7n0lp1r1yHYX...,https://api.spotify.com/v1/audio-analysis/7n0l...,211146,4


In [48]:
ninth_playlist = sp.user_playlist_tracks("Jimmy Alexander", "6aCHA5HeRfz5gJjllGZ7Of")

In [49]:
def get_tracks(user_id, playlist_id):
    results = sp.user_playlist_tracks(user_id, playlist_id)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks
ninth_results = get_tracks("Jimmy Alexander", "6aCHA5HeRfz5gJjllGZ7Of")


In [50]:
song_name = []
song_uri = []
song_id = []
song_artists = []
playlist = ninth_results
for r in ninth_results:
    song_name.append(r['track']['name'])
    song_uri.append(r['track']['uri'])
    song_id.append(r['track']['id'])
    song_artists.append(r['track']['artists'][0]['name'])
playlist9_info = pd.DataFrame({'song_name':song_name, 'song_id':song_id, 'artist':song_artists})



In [51]:

features9_info = pd.DataFrame(sp.audio_features(tracks = song_id[0:99]))
features9_info2 = pd.DataFrame(sp.audio_features(tracks = song_id[99:199]))
features9_info3 = pd.DataFrame(sp.audio_features(tracks = song_id[199:299]))
features9_info4 = pd.DataFrame(sp.audio_features(tracks = song_id[299:399]))
features9_info5 = pd.DataFrame(sp.audio_features(tracks = song_id[399:499]))
features9_info6 = pd.DataFrame(sp.audio_features(tracks = song_id[499:599]))
features9_info7 = pd.DataFrame(sp.audio_features(tracks = song_id[599:699]))
features9_info8 = pd.DataFrame(sp.audio_features(tracks = song_id[699:799]))
features9_info9 = pd.DataFrame(sp.audio_features(tracks = song_id[799:899]))
features9_info10 = pd.DataFrame(sp.audio_features(tracks = song_id[899:999]))
features9_info11 = pd.DataFrame(sp.audio_features(tracks = song_id[999:]))
features9 = pd.concat([features9_info, features9_info2, features9_info3, features9_info4, features9_info5, features9_info6, features9_info7, features9_info8, features9_info9, features9_info10, features9_info11])
features9.reset_index(drop=True, inplace=True)


In [52]:
ninth_playlist = pd.concat([playlist9_info, features9], axis=1)
ninth_playlist.reset_index()



,index,song_name,song_id,artist,danceability,energy,key,loudness,mode,speechiness,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0,'O Sole Mio,0oRa6dxPCzotYzujp21QFK,Enrico Caruso,0.287,0.149,7,-13.503,1,0.0379,...,0.2060,0.269,129.377,audio_features,0oRa6dxPCzotYzujp21QFK,spotify:track:0oRa6dxPCzotYzujp21QFK,https://api.spotify.com/v1/tracks/0oRa6dxPCzot...,https://api.spotify.com/v1/audio-analysis/0oRa...,204293,4
1,1,St. Louis Blues (with Louis Armstrong),4ejUoPIMggFTVrAyvodqJQ,Bessie Smith,0.408,0.145,10,-10.990,1,0.0501,...,0.1380,0.294,68.800,audio_features,4ejUoPIMggFTVrAyvodqJQ,spotify:track:4ejUoPIMggFTVrAyvodqJQ,https://api.spotify.com/v1/tracks/4ejUoPIMggFT...,https://api.spotify.com/v1/audio-analysis/4ejU...,190107,4
2,2,Allons A Lafayette,3zZyFcJZ84mjQWLNrtMr2Z,Joe Falcon & Cleoma Breaux,0.533,0.405,7,-14.048,1,0.0377,...,0.3930,0.868,93.960,audio_features,3zZyFcJZ84mjQWLNrtMr2Z,spotify:track:3zZyFcJZ84mjQWLNrtMr2Z,https://api.spotify.com/v1/tracks/3zZyFcJZ84mj...,https://api.spotify.com/v1/audio-analysis/3zZy...,176493,4
3,3,Lágrimas Negras,5UmrMwj8Ouw6yeg3TxQ9GA,Trio Matamoros,0.613,0.305,0,-10.041,0,0.2190,...,0.1110,0.789,96.967,audio_features,5UmrMwj8Ouw6yeg3TxQ9GA,spotify:track:5UmrMwj8Ouw6yeg3TxQ9GA,https://api.spotify.com/v1/tracks/5UmrMwj8Ouw6...,https://api.spotify.com/v1/audio-analysis/5Umr...,174520,4
4,4,St. James Infirmary,7fAa9rz4UmwuB4AGh50Gmp,Louis Armstrong & His Savoy Ballroom Five,0.693,0.182,5,-11.022,0,0.0514,...,0.1530,0.588,116.508,audio_features,7fAa9rz4UmwuB4AGh50Gmp,spotify:track:7fAa9rz4UmwuB4AGh50Gmp,https://api.spotify.com/v1/tracks/7fAa9rz4Umwu...,https://api.spotify.com/v1/audio-analysis/7fAa...,191867,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1028,1028,Tilted,4U6GYrZi0xvNkUmFitCDED,Christine and the Queens,0.872,0.361,7,-9.880,1,0.0609,...,0.1080,0.489,100.001,audio_features,4U6GYrZi0xvNkUmFitCDED,spotify:track:4U6GYrZi0xvNkUmFitCDED,https://api.spotify.com/v1/tracks/4U6GYrZi0xvN...,https://api.spotify.com/v1/audio-analysis/4U6G...,233247,4
1029,1029,Million Reasons,7dZ1Odmx9jWIweQSatnRqo,Lady Gaga,0.666,0.423,0,-8.012,1,0.0430,...,0.1060,0.154,129.890,audio_features,7dZ1Odmx9jWIweQSatnRqo,spotify:track:7dZ1Odmx9jWIweQSatnRqo,https://api.spotify.com/v1/tracks/7dZ1Odmx9jWI...,https://api.spotify.com/v1/audio-analysis/7dZ1...,205280,4
1030,1030,Lord,09CViXNGyH05AItXMtM6fC,Young Fathers,0.226,0.328,10,-9.804,1,0.0330,...,0.0851,0.119,175.558,audio_features,09CViXNGyH05AItXMtM6fC,spotify:track:09CViXNGyH05AItXMtM6fC,https://api.spotify.com/v1/tracks/09CViXNGyH05...,https://api.spotify.com/v1/audio-analysis/09CV...,217120,3
1031,1031,STREET LIVIN',6NjcOqkCcsbO65KhM5Af2K,Black Eyed Peas,0.693,0.730,4,-4.270,0,0.2210,...,0.3820,0.592,99.633,audio_features,6NjcOqkCcsbO65KhM5Af2K,spotify:track:6NjcOqkCcsbO65KhM5Af2K,https://api.spotify.com/v1/tracks/6NjcOqkCcsbO...,https://api.spotify.com/v1/audio-analysis/6Njc...,185413,4


In [53]:
tenth_playlist = sp.user_playlist_tracks("AROL", "20JT4SnBRiiXBeIXr5f2c0")

In [54]:
def get_tracks(user_id, playlist_id):
    results = sp.user_playlist_tracks(user_id, playlist_id)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks
tenth_results = get_tracks("AROL", "20JT4SnBRiiXBeIXr5f2c0")


In [55]:
song_name = []
song_uri = []
song_id = []
song_artists = []
playlist = tenth_results
for r in tenth_results:
    song_name.append(r['track']['name'])
    song_uri.append(r['track']['uri'])
    song_id.append(r['track']['id'])
    song_artists.append(r['track']['artists'][0]['name'])
playlist10_info = pd.DataFrame({'song_name':song_name, 'song_id':song_id, 'artist':song_artists})


TypeError: 'NoneType' object is not subscriptable

In [ ]:
features10_info = pd.DataFrame(sp.audio_features(tracks = song_id[0:99]))
features10_info2 = pd.DataFrame(sp.audio_features(tracks = song_id[99:199]))
features10_info3 = pd.DataFrame(sp.audio_features(tracks = song_id[199:299]))
features10_info4 = pd.DataFrame(sp.audio_features(tracks = song_id[299:399]))
features10_info5 = pd.DataFrame(sp.audio_features(tracks = song_id[399:499]))
features10_info6 = pd.DataFrame(sp.audio_features(tracks = song_id[499:599]))
features10_info7 = pd.DataFrame(sp.audio_features(tracks = song_id[599:699]))
features10_info8 = pd.DataFrame(sp.audio_features(tracks = song_id[699:799]))
features10_info9 = pd.DataFrame(sp.audio_features(tracks = song_id[799:899]))
features10_info10 = pd.DataFrame(sp.audio_features(tracks = song_id[899:999]))
features10_info11 = pd.DataFrame(sp.audio_features(tracks = song_id[999:1099]))
features10_info12 = pd.DataFrame(sp.audio_features(tracks = song_id[1099:1199]))
features10_info13 = pd.DataFrame(sp.audio_features(tracks = song_id[1199:1299]))
features10_info14 = pd.DataFrame(sp.audio_features(tracks = song_id[1299:1399]))
features10_info15 = pd.DataFrame(sp.audio_features(tracks = song_id[1399:1499]))
features10_info16 = pd.DataFrame(sp.audio_features(tracks = song_id[1499:1599]))
features10_info17 = pd.DataFrame(sp.audio_features(tracks = song_id[1599:1699]))
features10_info18 = pd.DataFrame(sp.audio_features(tracks = song_id[1699:1799]))
features10_info19 = pd.DataFrame(sp.audio_features(tracks = song_id[1799:1899]))
features10_info20 = pd.DataFrame(sp.audio_features(tracks = song_id[1899:1999]))
features10_info21 = pd.DataFrame(sp.audio_features(tracks = song_id[1999:2099]))
features10_info22 = pd.DataFrame(sp.audio_features(tracks = song_id[2099:2199]))
features10_info23 = pd.DataFrame(sp.audio_features(tracks = song_id[2199:2299]))
features10_info24 = pd.DataFrame(sp.audio_features(tracks = song_id[2299:2399]))
features10_info25 = pd.DataFrame(sp.audio_features(tracks = song_id[2399:2499]))
features10_info26 = pd.DataFrame(sp.audio_features(tracks = song_id[2499:2599]))
features10_info27 = pd.DataFrame(sp.audio_features(tracks = song_id[2599:2699]))
features10_info28 = pd.DataFrame(sp.audio_features(tracks = song_id[2699:2799]))
features10_info29 = pd.DataFrame(sp.audio_features(tracks = song_id[2799:2899]))
features10_info30 = pd.DataFrame(sp.audio_features(tracks = song_id[2899:2999]))
features10_info31 = pd.DataFrame(sp.audio_features(tracks = song_id[2999:3099]))
features10_info32 = pd.DataFrame(sp.audio_features(tracks = song_id[3099:3199]))
features10_info33 = pd.DataFrame(sp.audio_features(tracks = song_id[3199:3299]))
features10_info34 = pd.DataFrame(sp.audio_features(tracks = song_id[3299:3399]))
features10_info35 = pd.DataFrame(sp.audio_features(tracks = song_id[3399:3499]))
features10_info36 = pd.DataFrame(sp.audio_features(tracks = song_id[3499:3599]))
features10_info37 = pd.DataFrame(sp.audio_features(tracks = song_id[3599:3699]))
features10_info38 = pd.DataFrame(sp.audio_features(tracks = song_id[3699:3799]))
features10_info39 = pd.DataFrame(sp.audio_features(tracks = song_id[3799:3899]))
features10_info40 = pd.DataFrame(sp.audio_features(tracks = song_id[3899:3999]))
features10_info41 = pd.DataFrame(sp.audio_features(tracks = song_id[3999:4099]))
features10_info42 = pd.DataFrame(sp.audio_features(tracks = song_id[4099:4199]))
features10_info43 = pd.DataFrame(sp.audio_features(tracks = song_id[4199:4299]))
features10_info44 = pd.DataFrame(sp.audio_features(tracks = song_id[4299:4399]))
features10_info45 = pd.DataFrame(sp.audio_features(tracks = song_id[4399:4499]))
features10_info46 = pd.DataFrame(sp.audio_features(tracks = song_id[4499:4599]))
features10_info47 = pd.DataFrame(sp.audio_features(tracks = song_id[4599:]))
features10 = pd.concat([features9_info, features10_info2, features10_info3, features10_info4, 
                        features10_info5, features10_info6, features10_info7, features10_info8, 
                        features10_info9, features10_info10, features10_info11, features10_info12, 
                        features10_info13, features10_info14, features10_info15, features10_info16, 
                        features10_info17, features10_info18, features10_info19, features10_info20, 
                        features10_info21, features10_info22, features10_info23, features10_info24, 
                        features10_info25, features10_info26, features10_info27, features10_info28, 
                        features10_info29, features10_info30, features10_info31, features10_info32, 
                        features10_info33, features10_info34, features10_info35, features10_info36, 
                        features10_info37, features10_info38, features10_info39, features10_info40, 
                        features10_info41, features10_info42, features10_info43, features10_info44, 
                        features10_info45, features10_info46, features10_info47])
features10.reset_index(drop=True, inplace=True)


In [ ]:
tenth_playlist = pd.concat([playlist10_info, features10], axis=1)
tenth_playlist.reset_index()


In [ ]:
huge_playlist = pd.concat([first_playlist, second_playlist, third_playlist, fourth_playlist, 
                           fifth_playlist, sixth_playlist, seventh_playlist, eighth_playlist, 
                           ninth_playlist, tenth_playlist])
huge_playlist.reset_index(drop=True, inplace=True)


In [ ]:
huge_playlist.duplicated(subset='song_name', keep='first').sum()
huge_playlist = huge_playlist.drop_duplicates()
huge_playlist.reset_index(drop=True, inplace=True)
huge_playlist = huge_playlist.dropna()
huge_playlist.reset_index(drop=True, inplace=True)


In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import plotly 
import plotly.graph_objs as go
from sklearn.cluster import KMeans 
from sklearn import datasets 
from sklearn.preprocessing import StandardScaler 



In [ ]:
# copying the playlist just in case
hugeplaylist = huge_playlist.copy()


In [ ]:
# dropping extra columns, leaving only the song to use later as index
hugeplaylist.drop(['artist', 'song_id', 'id', 'uri', 'track_href', 'mode', 'analysis_url', 
                   'duration_ms', 'time_signature', 'type'], axis = 1, inplace = True)


In [ ]:
# Song as index!
hugeplaylist.index = hugeplaylist.iloc[:,0]
# pick first column and turn it to index [:,_] - location



In [ ]:
#keep rest of columns
hugeplaylist = hugeplaylist.iloc[:,1:]

In [ ]:

# remove the header name
hugeplaylist.rename_axis(None,inplace=True)


In [ ]:
hugeplaylist_scaled = StandardScaler().fit_transform(hugeplaylist)


In [ ]:
hugeplaylist_scaled_df = pd.DataFrame(hugeplaylist_scaled,columns = ['danceability', 'energy', 
                                                                     'key', 'loudness', 
                                                                     'speechiness', 'acousticness', 
                                                                     'instrumentalness', 'liveness', 
                                                                     'valence', 'tempo'])


In [ ]:

kmeans = KMeans(n_clusters = 52)


In [ ]:
cluster = kmeans.fit(hugeplaylist_scaled_df)

In [ ]:
#bring cluster into data frame
hugeplaylist_c = hugeplaylist.copy()
hugeplaylist_c['cluster'] = cluster.labels_


In [ ]:
### Adding the cluster column to the original dataframe and reoganizing the data


# In[72]:


huge_playlist['cluster'] = hugeplaylist_c['cluster'].values


In [ ]:
# creating the cluster thing
from IPython.core.display import display
from IPython.display import IFrame



In [ ]:
def song_cluster(song):
    song = sp.search(q = song, limit = 1)
    song_id = song['tracks']
    song_id = song_id['items'][0]['id']
    song_features = sp.audio_features(song_id)
    song_data = pd.DataFrame(song_features, index= [0])
   # droping the numeric columns i didn't use
    song_data.drop(['mode', 'duration_ms', 'time_signature'], axis = 1, inplace = True)  
    song_num = song_data._get_numeric_data()
    song_scaled = StandardScaler().fit_transform(song_num)
    cluster = kmeans.predict(song_scaled)
    song_cluster = huge_playlist.loc[huge_playlist['cluster'] == int(cluster)]
    song_cluster.reset_index(drop=True, inplace=True) 
    song_name_id = random.choice(song_cluster["id"])
    songname = sp.track(song_name_id)['name']
    artist = sp.track(song_name_id)['artists'][0]['name']
    def recommend(song_name_id):
        display(IFrame(src=f"https://open.spotify.com/embed/track/{song_name_id}",
                        width="320",
                        height="80",
                        frameborder="0",
                        allowtransparency="true",
                        allow="encrypted-media",))
    return print("\033[1m" + 'That song is not hot right now. But listen to ' + songname + ' by ' + artist + ". I think you'll like it!."), recommend(song_name_id)

In [ ]:
##### practice other ways

In [ ]:
# new suggester attempt

def second_suggester():
    song = str(input( "\033[1m" + """
    Welcome to the awesome song recommender!
    What song do you have in mind?: 
    """).lower().replace(" ", ''))
    # check hot 100
    check = billboard_top100[billboard_top100['song'].str.lower().str.replace(" ","").str.contains(song)]
    # get index
    index = check.index.tolist()
    if len(check) == 0:
        song_cluster(song)
        song2 = str(input("\033[1m" + "Want to try again? [y/n]: ")).lower().replace(" ", '')
        if song2 == 'y':
            print("\033[1m" + 'Awesome! Here, have another go:')
            second_suggester()
        elif song2 != 'n':
            print("\033[1m" + "Oh, come on, that's not an answer")
        elif song2 == 'n':
            print("\033[1m" + 'Oh well, your bad, this is an awesome recommender')
            return
    else:
        answer1 = input("\033[1m" + "Are you thinking about '" + billboard_top100.song[index].values[0] + "' by '" + billboard_top100.artist[index].values[0] + "'? [y/n]: ")
        #make a song suggestion
        if answer1.lower() == 'y':
            suggestion = billboard_top100.sample().index.tolist()
            print("\033[1m" + "That's a hot song! Listen to '" + billboard_top100['song'][suggestion].item() + "' by '" + billboard_top100['artist'][suggestion].item() + "' I think you'll like it.")
            song2 = str(input("\033[1m" + "Want to try again? [y/n]: ")).lower().replace(" ", '')
            if song2 == 'y':
                print("\033[1m" + 'Awesome! Here, have another go:')
                second_suggester()
            elif song2 != 'n':
                print("\033[1m" + "Oh, come on, that's not an answer")
            elif song2 == 'n':
                print("\033[1m" + 'Oh well, your bad, this is an awesome recommender')
                return
        elif answer1.lower() != 'n':
            print("\033[1m" + "Oh, come on, that's not an answer")
            song2 = str(input("\033[1m" + "Want to try again? [y/n]: ")).lower().replace(" ", '')
            second_suggester()
        elif answer1.lower() == 'n':
            song_cluster(song)
            song2 = str(input("\033[1m" + "Want to try again? [y/n]: ")).lower().replace(" ", '')
            if song2 == 'y':
                print("\033[1m" + 'Awesome! Here, have another go:')
                second_suggester()
            elif song2 != 'n':
                print("\033[1m" + "Oh, come on, that's not an answer")
            elif song2 == 'n':
                print("\033[1m" + 'Oh well, your bad, this is an awesome recommender')
                return
        elif song2 != 'n':
            print("\033[1m" + "Oh, come on, that's not an answer")
        elif song2 == 'n':
            print("\033[1m" + 'Oh well, your bad, this is an awesome recommender')
            return
second_suggester()
